<a href="https://colab.research.google.com/github/seecode4/seeRepo1/blob/main/Student_MLE_MiniProject_Fine_Tuning_m3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.backend import resize_images
from sklearn.model_selection import train_test_split

from tensorflow import keras as K
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.layers import Input, Lambda, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.initializers import RandomNormal, RandomUniform

In [ ]:
# Load the CIFAR-10 dataset
cifar_home = 'https://www.cs.toronto.edu/~kriz/cifar.html'
# from tensorflow.keras.datasets import cifar10
%cd /content
!pwd
!rm -fr cifar*
!ls
print('---------')
(x_train_val, y_train_val), (x_test, y_test) = cifar10.load_data()
assert x_train_val.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train_val.shape == (50000, 1)
assert y_test.shape == (10000, 1)
print(f'x_train_val type: {type(x_train_val)}, ndim:{x_train_val.ndim}, shape:{x_train_val.shape}')
print(f'{x_train_val[0][0][0:10]}')

/content
/content
sample_data
---------
170498071/170498071 [==============================] - 2s 0us/step
x_train_val type: <class 'numpy.ndarray'>, ndim:4, shape:(50000, 32, 32, 3)
[[ 59  62  63]
 [ 43  46  45]
 [ 50  48  43]
 [ 68  54  42]
 [ 98  73  52]
 [119  91  63]
 [139 107  75]
 [145 110  80]
 [149 117  89]
 [149 120  93]]


In [ ]:
# One-hot encode the labels - with to_categorical
num_classes = 10
y_train_val_1hot = to_categorical(y_train_val, num_classes)
y_test_1hot= to_categorical(y_test, num_classes)
print(y_train_val_1hot.shape, y_test_1hot.shape)

(50000, 10) (10000, 10)


In [ ]:
# Normalize the pixel values to [0, 1]
np.set_printoptions(precision=4)
X_train_val = x_train_val/255
print(f'X_train_val:\n {X_train_val[0][0][0:10]}...')
X_test = x_test/255
print(f'X_test:\n {X_test[0][0][0:10]}...')

X_train_val:
 [[0.2314 0.2431 0.2471]
 [0.1686 0.1804 0.1765]
 [0.1961 0.1882 0.1686]
 [0.2667 0.2118 0.1647]
 [0.3843 0.2863 0.2039]
 [0.4667 0.3569 0.2471]
 [0.5451 0.4196 0.2941]
 [0.5686 0.4314 0.3137]
 [0.5843 0.4588 0.349 ]
 [0.5843 0.4706 0.3647]]...
X_test:
 [[0.6196 0.4392 0.1922]
 [0.6235 0.4353 0.1843]
 [0.6471 0.4549 0.2   ]
 [0.651  0.4627 0.2078]
 [0.6275 0.4392 0.1804]
 [0.6118 0.4275 0.1608]
 [0.6353 0.451  0.1843]
 [0.6235 0.4431 0.1765]
 [0.6196 0.4353 0.1725]
 [0.6235 0.4431 0.1608]]...


In [ ]:
# Config to make results repeatable/deterministic
tf.config.experimental.enable_op_determinism()
SEED=42
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)

In [ ]:
base_model = VGG16(
    weights='imagenet',  # load weights pretrained on ImageNet
    include_top=False,   # exclude the top classifier
    input_shape=(32, 32, 3),
    pooling='max'        # add a global max pooling layer after each layer
)
# Freeze the the trainable param (weights and biases) so they are not retrained
for layer in base_model.layers:
   layer.trainable = False
# print(base_model.summary())

model3 = Sequential()
initializer = RandomNormal(mean=0.0, stddev=1.0, seed=SEED)
for layer in base_model.layers[:-1]:
    model3.add(layer)

# model3.add(Flatten())
model3.add(GlobalAveragePooling2D(data_format='channels_last'))
model3.add(Dense(256, activation="relu", kernel_initializer='he_normal'))

# Try to prevent overfitting
model3.add(tf.keras.layers.Dropout(.5))
model3.add(Dense(256, activation="relu", kernel_initializer='he_normal'))

# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation
model3.add(Dense(10, activation='softmax', kernel_initializer=initializer))

# Compile the model - use Adagrad optimizer
model3.compile(Adagrad(learning_rate=0.001, use_ema=True, ema_momentum=0.9),
               loss='categorical_crossentropy', metrics=['accuracy'])
print(model3.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0         
                                                                 
 block3_conv1 (Conv2D)       (None, 8, 8, 256)        

In [ ]:
# # Data Augmentation
# datagen_aug = ImageDataGenerator(rotation_range=20,
#                              width_shift_range=0.2,
#                              height_shift_range=0.2,
#                              horizontal_flip=True
#                              )
# datagen_aug.fit(X_train_val)

In [ ]:
%%time
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
# def lr_scheduler(epoch):
#     return 0.002 * (0.5 ** (epoch // 20))
# reduce_lr = LearningRateScheduler(lr_scheduler)

# Define a callback to monitor val_accuracy
monitor_acc = EarlyStopping(monitor='accuracy',
                       min_delta=0.001, patience=4)

# Train the model
print(X_train_val.shape, y_train_val_1hot.shape)
model3.fit(X_train_val, y_train_val_1hot, epochs=50, batch_size=64,
          validation_split=0.2, validation_freq=2, verbose=1,
          callbacks= [monitor_acc])

(50000, 32, 32, 3) (50000, 10)
Epoch 1/50
625/625 [==============================] - 34s 53ms/step - loss: 4.2886 - accuracy: 0.1921
Epoch 2/50
625/625 [==============================] - 42s 67ms/step - loss: 2.3882 - accuracy: 0.2560 - val_loss: 1.8594 - val_accuracy: 0.3371
Epoch 3/50
625/625 [==============================] - 34s 54ms/step - loss: 2.1282 - accuracy: 0.2839
Epoch 4/50
625/625 [==============================] - 41s 66ms/step - loss: 2.0281 - accuracy: 0.2982 - val_loss: 1.7654 - val_accuracy: 0.3761
Epoch 5/50
625/625 [==============================] - 34s 54ms/step - loss: 1.9715 - accuracy: 0.3154
Epoch 6/50
625/625 [==============================] - 41s 66ms/step - loss: 1.9181 - accuracy: 0.3304 - val_loss: 1.7091 - val_accuracy: 0.4055
Epoch 7/50
625/625 [==============================] - 34s 54ms/step - loss: 1.8769 - accuracy: 0.3443
Epoch 8/50
625/625 [==============================] - 41s 66ms/step - loss: 1.8540 - accuracy: 0.3553 - val_loss: 1.6650 - val_ac

In [ ]:
# Train the model
print(X_train_val.shape, y_train_val_1hot.shape)
model3.fit(X_train_val, y_train_val_1hot, epochs=50, batch_size=64,
          validation_split=0.2, validation_freq=2, verbose=1,
          callbacks= [monitor_acc])

(50000, 32, 32, 3) (50000, 10)
Epoch 1/50
625/625 [==============================] - 34s 54ms/step - loss: 1.5473 - accuracy: 0.4580
Epoch 2/50
625/625 [==============================] - 43s 68ms/step - loss: 1.5437 - accuracy: 0.4582 - val_loss: 1.4190 - val_accuracy: 0.5032
Epoch 3/50
625/625 [==============================] - 34s 55ms/step - loss: 1.5450 - accuracy: 0.4582
Epoch 4/50
625/625 [==============================] - 42s 67ms/step - loss: 1.5425 - accuracy: 0.4590 - val_loss: 1.4142 - val_accuracy: 0.5050
Epoch 5/50
625/625 [==============================] - 34s 54ms/step - loss: 1.5322 - accuracy: 0.4627
Epoch 6/50
625/625 [==============================] - 42s 67ms/step - loss: 1.5321 - accuracy: 0.4651 - val_loss: 1.4101 - val_accuracy: 0.5080
Epoch 7/50
625/625 [==============================] - 34s 54ms/step - loss: 1.5300 - accuracy: 0.4601
Epoch 8/50
625/625 [==============================] - 42s 67ms/step - loss: 1.5267 - accuracy: 0.4643 - val_loss: 1.4071 - val_ac

In [ ]:
# Train the model
print(X_train_val.shape, y_train_val_1hot.shape)
model3.fit(X_train_val, y_train_val_1hot, epochs=50, batch_size=64,
          validation_split=0.2, validation_freq=2, verbose=1,
          callbacks= [monitor_acc])

(50000, 32, 32, 3) (50000, 10)
Epoch 1/50
625/625 [==============================] - 34s 54ms/step - loss: 1.5233 - accuracy: 0.4658
Epoch 2/50
625/625 [==============================] - 42s 68ms/step - loss: 1.5254 - accuracy: 0.4651 - val_loss: 1.3999 - val_accuracy: 0.5118
Epoch 3/50
625/625 [==============================] - 34s 54ms/step - loss: 1.5164 - accuracy: 0.4688
Epoch 4/50
625/625 [==============================] - 42s 68ms/step - loss: 1.5172 - accuracy: 0.4682 - val_loss: 1.3972 - val_accuracy: 0.5133
Epoch 5/50
625/625 [==============================] - 34s 54ms/step - loss: 1.5128 - accuracy: 0.4699
Epoch 6/50
625/625 [==============================] - 42s 67ms/step - loss: 1.5119 - accuracy: 0.4699 - val_loss: 1.3935 - val_accuracy: 0.5121
Epoch 7/50
625/625 [==============================] - 34s 54ms/step - loss: 1.5060 - accuracy: 0.4719
Epoch 8/50
625/625 [==============================] - 42s 67ms/step - loss: 1.5096 - accuracy: 0.4733 - val_loss: 1.3906 - val_ac

In [ ]:
# Train the model
print(X_train_val.shape, y_train_val_1hot.shape)
model3.fit(X_train_val, y_train_val_1hot, epochs=50, batch_size=1024,
          validation_split=0.2, validation_freq=2, verbose=1,
          callbacks= [monitor_acc])

(50000, 32, 32, 3) (50000, 10)
Epoch 1/50
40/40 [==============================] - 16s 381ms/step - loss: 1.4877 - accuracy: 0.4777
Epoch 2/50
40/40 [==============================] - 19s 478ms/step - loss: 1.4828 - accuracy: 0.4793 - val_loss: 1.3765 - val_accuracy: 0.5205
Epoch 3/50
40/40 [==============================] - 15s 377ms/step - loss: 1.4881 - accuracy: 0.4778
Epoch 4/50
40/40 [==============================] - 19s 474ms/step - loss: 1.4840 - accuracy: 0.4813 - val_loss: 1.3763 - val_accuracy: 0.5205
Epoch 5/50
40/40 [==============================] - 15s 373ms/step - loss: 1.4906 - accuracy: 0.4774
Epoch 6/50
40/40 [==============================] - 19s 479ms/step - loss: 1.4840 - accuracy: 0.4790 - val_loss: 1.3762 - val_accuracy: 0.5208
Epoch 7/50
40/40 [==============================] - 15s 382ms/step - loss: 1.4871 - accuracy: 0.4799
Epoch 8/50
40/40 [==============================] - 19s 478ms/step - loss: 1.4861 - accuracy: 0.4805 - val_loss: 1.3763 - val_accuracy: 

In [ ]:
# Evaluate the model on the test set
from sklearn.metrics import roc_auc_score, confusion_matrix

y_test_pred = model3.predict(X_test)
print(y_test_1hot.shape, y_test_pred.shape)

313/313 [==============================] - 11s 34ms/step
(10000, 10) (10000, 10)


In [ ]:
test_score = roc_auc_score(y_test_1hot, y_test_pred)
print("roc_auc_score:", test_score)

y_test_pred_val = np.argmax(y_test_pred, axis=1).reshape(-1,)
y_test_val = np.ravel(y_test)
print(y_test_val.shape, y_test_pred_val.shape)
print(type(y_test), type(y_test_pred_val))

cm = confusion_matrix(y_test_val, y_test_pred_val)
print(cm)

roc_auc_score: 0.8899618222222221
(10000,) (10000,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[[542  40 104  33  34   6  13  21 141  66]
 [ 23 577  19  39  13  18  35  26  47 203]
 [ 69  32 405  61 189  48 114  38  21  23]
 [ 12  69  86 313 133 107 122  39  22  97]
 [ 18  17  72  43 621  20  69  71  36  33]
 [  9  38 110 195 110 347  53  85   9  44]
 [  8  38  61  68 148  30 601   6  13  27]
 [ 13  22  74  53 166  46  12 495  16 103]
 [ 82  83  19  18  38   2  11   6 644  97]
 [ 32 156  14  38  35   8  23  40  57 597]]


Summary in Student_MLE_MiniProject_Fine_Tuning_m3a.ipynb